## Image Classification Recap
We will explore the use of Google Colab by running some code that we are familiar with. We will re-run the code from the last practical session, with and without a GPU. Let's start off by importing the necessary libraries.

In [13]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.losses import categorical_crossentropy
from keras import backend as K

The remainder of this code is similar to last week's session, if you are unsure what something does, you can review it in last week's notebook. This week however, we will use all 60000 images to train the network, rather than take a subsample of 5000 images. This is because can now use a GPU.

In [14]:
num_classes = 10
batch_size = 128
epochs = 12
img_rows, img_cols = 28, 28

In [15]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [16]:
# add channels
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [17]:
# change the data type and send pixels to [0,1]
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 
x_test /= 255

In [18]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### Building the model
We're going to build a similar model to last time, but we will increase the depth and width of the network. We do this by adding more layers, and changing the number of channels in each layer.

In [22]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Flatten()) # This line is to convert from matrices to vectors
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax')) # we are working with vectors now, so we use a Dense layer instead of Conv2d
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 12, 12, 64)        256       
_________________________________________________________________
dropout_5 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 10, 128)       73856     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 128)        

In [23]:
model.compile(loss=categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

### Training the model
We train the model in the same way as last week, with the model.fit function.

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
 1152/60000 [..............................] - ETA: 22:01 - loss: 1.7792 - accuracy: 0.4375

This is really slow. To speed up the performance, switch to a GPU. To do this go to
    `Edit -> Notebook settings -> Hardware accelerator -> GPU`
and re-run the notebook.

Instead of having to wait around an hour, we now only have to wait a couple of minutes. Let's take a look to see our final results.

In [ ]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Last week we got an accuracy of around 96%. Now, because we are using more data and a deeper, wider network, we can significantly reduce this error to within 1%.